In [1]:
import sys
print(sys.version)

3.8.12 (default, Oct 12 2021, 08:03:51) [MSC v.1916 32 bit (Intel)]


In [2]:
import win32com.client
import time
import shutil
import os
import codecs
import pandas as pd
import numpy as np
from datetime import datetime
from tqdm import tqdm
# from pywinauto import application
# from pywinauto import timings


class CREON(object):
    """대신증권 크레온 API"""

    def __init__(self):
        # 연결 여부 체크
        self.objCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
        bConnect = self.objCpCybos.IsConnect
        if (bConnect == 0):
            print("PLUS가 정상적으로 연결되지 않음. ")
        else:
#             print("정상적으로 연결되었음")
            pass

    def setMethod(self, code, char, minutes=1, from_yyyymmdd=None, to_yyyymmdd=None, count=None):
        """
        count는 보통 상식의 데이터 개수가 아니다.
        여기서는 한번 요청 시 가져와지는 데이터의 개수이다.
        한번 요청 시 최대 2856개 가능하다.

        원하는 데이터 개수가 있으면 to_yyyymmdd 로 가져온 다음에 잘라서 사용한다.
        하루에 분단위 데이터가 381개이다. (* 마지막 10분은 동시호가)

        """
        # object 구하기
        self.objStockChart = win32com.client.Dispatch("CpSysDib.StockChart")
        self.objStockChart.SetInputValue(0, code)  # 종목코드

        if to_yyyymmdd:
            self.objStockChart.SetInputValue(1, ord('1'))  # 요청 구분 '1': 기간, '2': 개수
            self.objStockChart.SetInputValue(2, from_yyyymmdd)  # To 날짜
            self.objStockChart.SetInputValue(3, to_yyyymmdd)  # From 날짜
        elif count:
            self.objStockChart.SetInputValue(1, ord('2'))  # 개수로 받기
            self.objStockChart.SetInputValue(4, count)  # 조회 개수
        else: 
            raise print("기간을 입력해주세요.")

        # 날짜, 시간,시가,고가,저가,종가,거래량
        self.colnames = "날짜, 시간, 시가, 고가, 저가, 종가, 거래량, 거래대금".split(", ")
        self.objStockChart.SetInputValue(5, [0, 1, 2, 3, 4, 5, 8, 9])
        # 날짜,시가,고가,저가,종가,거래량, 거래대금, 상장주식수, 시가총액, 외국인현보유수량, 기관순매수
#         self.colnames = "날짜, 시가, 고가, 저가, 종가, 거래량, 거래대금, 상장주식수, 시가총액".split(", ")
#         self.objStockChart.SetInputValue(5, [0, 2, 3, 4, 5, 8, 9, 12, 13])

        self.objStockChart.SetInputValue(6, ord(char))  # '차트 주기 - 분/틱 # # 'D', m'
        if char == "m":
            self.objStockChart.SetInputValue(7, minutes)  # 분틱차트 주기 ## 1분은 1로

        self.objStockChart.SetInputValue(9, ord('1'))  # 수정주가 사용

        self.data = {i: [] for i in self.colnames}

    def checkRequest(self):
        self.objStockChart.BlockRequest()
        rqStatus = self.objStockChart.GetDibStatus()
        if rqStatus != 0: 
            return False
        self.count = self.objStockChart.GetHeaderValue(3)
        if self.count <= 1: 
            return False
        return int(self.count)

    def checkRemainTime(self):
        # 연속 요청 가능 여부 체크
        remainTime = self.objCpCybos.LimitRequestRemainTime / 1000.
        remainCount = self.objCpCybos.GetLimitRemainCount(1)  # 시세 제한
#         print("남은시간: ",remainTime, " 남은개수: ", remainCount)
        if remainCount <= 0:
#             print("15초당 60건으로 제한합니다.")
            time.sleep(remainTime)


    def getStockPriceMin(self):
        while 1:
            self.checkRemainTime()
            rows = self.checkRequest()
            if rows:
                for i in range(rows):
                    for idx, col in enumerate(self.colnames):
                        self.data[col].append(self.objStockChart.GetDataValue(idx, i))
            else:
                break
        self.data = pd.DataFrame(self.data).astype('str')
        return self.data

In [3]:
code = 'A005930'
name='삼성전자'
creon = CREON()
creon.setMethod(code=code, char="m", minutes=5, to_yyyymmdd=20000101,from_yyyymmdd=20221126)
# creon.setMethod(code=code, char="D", to_yyyymmdd=20200519,from_yyyymmdd=20200520)
getStockPrice = creon.getStockPriceMin()
print('---------name : {} 요청은 완료됬습니다.'.format(name))

---------name : 삼성전자 요청은 완료됬습니다.


In [4]:
getStockPrice

,날짜,시간,시가,고가,저가,종가,거래량,거래대금
0,20221125,1530,61000,61000,61000,61000,541347,33022160000
1,20221125,1520,61000,61100,60900,60900,219911,13415910000
2,20221125,1515,61000,61100,61000,61100,72973,4453340000
3,20221125,1510,61100,61100,61000,61000,69764,4259960000
4,20221125,1505,61100,61100,61000,61000,42247,2579040000
...,...,...,...,...,...,...,...,...
95559,20171107,925,56440,56460,56240,56320,69850,3935620000
95560,20171107,920,56400,56460,56300,56440,117250,6613180000
95561,20171107,915,56240,56400,56220,56400,101150,5700380000
95562,20171107,910,56140,56400,56140,56240,189850,10691470000


In [4]:
data_all = getStockPrice
data_all.rename(columns={'날짜':'date','시간':'time','시가':'open','고가':'high','저가':'low','종가':'close','거래량':'volume','거래대금':'value'},inplace=True)
data_all['date'] = [datetime.strptime(str(d)+str(t).zfill(4),'%Y%m%d%H%M')  for d, t in zip(data_all.date, data_all.time)]
data_all = data_all[['date','open','high','low','close','volume','value']]
data_all

,date,open,high,low,close,volume,value
0,2020-05-20 15:30:00,50000,50000,50000,50000,1027059,51352950000
1,2020-05-20 15:20:00,49900,49950,49850,49900,604623,30174270000
2,2020-05-20 15:10:00,49900,49950,49900,49900,286981,14324320000
3,2020-05-20 15:00:00,49950,50000,49900,49900,525816,26270080000
4,2020-05-20 14:50:00,50000,50000,49950,50000,239258,11958520000
...,...,...,...,...,...,...,...
73,2020-05-19 09:50:00,49950,49950,49800,49900,891391,44468770000
74,2020-05-19 09:40:00,50000,50000,49900,49900,768793,38408110000
75,2020-05-19 09:30:00,50000,50100,49950,50000,763412,38160290000
76,2020-05-19 09:20:00,49950,50200,49950,50000,1706957,85463190000


In [5]:
import FinanceDataReader as fdr
a = fdr.StockListing('KRX')
aa = a[(a.Market=='KOSPI')|(a.Market=='KOSDAQ')]
list_code = aa.Code

In [4]:
list_existing = os.listdir('./data/10m/')
list_existing = [ext[1:7] for ext in list_existing]

In [7]:
#     code = 'A000660'
for code_ in tqdm(list_code):
#     if code_ in list_existing:
#         continue
    code =f'A{code_}'
# name='삼성전자'
    try:
        creon = CREON()
        creon.setMethod(code=code, char="m", minutes=5, to_yyyymmdd=20000101,from_yyyymmdd=20221126)
        #creon.setMethod(code=code, char="D", to_yyyymmdd=20200519,from_yyyymmdd=20200520)
        getStockPrice = creon.getStockPriceMin()
    # print('---------name : {} 요청은 완료됬습니다.'.format(name))
        data_all = getStockPrice.copy()
        data_all.rename(columns={'날짜':'date','시간':'time','시가':'open','고가':'high','저가':'low','종가':'close','거래량':'volume','거래대금':'value','상장주식수':'shares','시가총액':'cap'},inplace=True)
#         display(data_all)
        data_all['date'] = [str(d)+str(t).zfill(4) for d, t in zip(data_all.date, data_all.time)]
        data_all = data_all[['date','time','open','high','low','close','volume','value']]
        data_all.to_csv(f'./data/5m/{code}.csv')
    except:
        pass

100%|███████████████████████████████████████████████████| 2504/2504 [5:20:45<00:00,  7.69s/it]


In [8]:
data_all

,date,open,high,low,close,volume,value
0,202211241530,86900,86900,86900,86900,211762,18402120000
1,202211241520,86900,87000,86800,86900,4457,387220000
2,202211241519,86900,87000,86800,86900,5479,475940000
3,202211241518,86900,87000,86900,86900,746,64850000
4,202211241517,87000,87000,86900,87000,21899,1903080000
...,...,...,...,...,...,...,...
189993,20201118905,100000,100500,100000,100500,41438,4152550000
189994,20201118904,100000,100500,99900,100000,61676,6179520000
189995,20201118903,99600,100000,99600,100000,65092,6501460000
189996,20201118902,99500,99600,99300,99600,33762,3358990000
